In [23]:
import pathlib
import subprocess
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [25]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Calculate statistics at key mutation sites in early sequences

8782, 28144, 29095


In [26]:
def write_text(out_file, text, file_opt='a'):
    try:
        f = open(out_file, file_opt)
        f.write(text)
        f.write('\n')
        f.close()
    except Exception as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [27]:
def remove_clipping(alignment_path, bam_file):
    #removes soft clipping from reads, keeps matched section
    out_file=bam_file.split('.bam')[0]+'_removedclip.bam'
    try:
        cmd = f"{NGSUTILS_PATH}ngsutilsj bam-removeclipping -f {alignment_path+bam_file} {alignment_path+out_file}"
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    #bam file needs to be indexed for bam-readcount
    try:
        cmd = f"{SAMTOOLS_PATH}samtools index {alignment_path+out_file}"
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    return out_file

In [28]:
def run_bam_readcount(alignment_path, bam_file, site_list, min_mapq=0, min_baseq=0):
    out_file=bam_file.split('.bam')[0]+'.brc.tsv'
    try:
        cmd = f"{BAM_READCOUNT_PATH}bam-readcount -w1 -f {REF_PATH+REF_NAME} {alignment_path+bam_file} {site_list} > {alignment_path+out_file}"
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    return out_file

In [29]:
def get_base_data(field_data, base_fields):
    # Iterate over each base/indel
    base_data_list=[]
    for base_data_string in field_data:
        #print(f'base_data_string: {base_data_string}')
        base_data = {}
        base_values = base_data_string.split(':')
        for i, base_field in enumerate(base_fields.keys()):
            base_data[base_field] = base_fields[base_field](base_values[i])
        base_data_list.append(base_data)
    return base_data_list

In [30]:
def bam_readcount_results(sra, bam_readcount_output,min_depth=0,min_vaf=0):
    base_fields = {
        'base': str,
        'count': int,
        'avg_mapping_quality': float,
        'avg_basequality': float,
        'avg_se_mapping_quality': float,
        'num_plus_strand': int,
        'num_minus_strand': int,
        'avg_pos_as_fraction': float,
        'avg_num_mismatches_as_fraction': float,
        'avg_sum_mismatch_qualities': float,
        'num_q2_containing_reads': int,
        'avg_distance_to_q2_start_in_q2_reads': float,
        'avg_clipped_length': float,
        'avg_distance_to_effective_3p_end': float
    }
    
    with open(bam_readcount_output) as in_fh:
        tsv_strings=[]
        for i, line in enumerate(in_fh):
            line = line.strip()
            fields = line.split('\t')
            # The first four fields contain overall information about the position
            chrom = fields[0]               # Chromosome/reference name
            position = int(fields[1])       # Position (1-based)
            reference_base = fields[2]      # Reference base
            depth = int(fields[3])          # Depth of coverage
            
            base_data_l=get_base_data(fields[4:], base_fields)

            # Skip zero-depth bases
            if depth == 0:
                print(chrom+'\t'+str(position)+'\t'+reference_base+'\t'+'zero depth')
                continue
            for base_data in base_data_l:
                # Calculate an allele frequency (VAF) from the base counts
                vaf = base_data['count'] / depth
                # Filter on minimum depth and VAF
                if depth >= min_depth and vaf >= min_vaf:
                    # Output count and VAF data as well as avg_pos_as_fraction
                    tsv_string='\t'.join(str(x) for x in (sra, chrom, position, reference_base, base_data['base'],'%0.2f' % (vaf), depth, base_data['count'],
                                                 base_data['avg_basequality'], base_data['avg_pos_as_fraction']))
                    
                    tsv_strings.append(tsv_string)
    return tsv_strings

In [31]:
def plot_vaf_vs_avg_pos(df, min_depth=0, min_var=0, min_vaf=0, fig_file=None):
    select = (df['depth'] >= min_depth) & \
    (df['count'] >= min_var) & \
    (df['vaf'] >= min_vaf)

    df = df[select]

    plt.plot(df['avg_pos_as_fraction'], df['vaf'], 'o', color='black')
    plt.title('vaf vs avg_pos_as_fraction')
    plt.xlabel('avg_pos_as_fraction')
    plt.ylabel('vaf')
    if fig_file:
        plt.savefig(fig_file)
    plt.show()
        

In [32]:
def plot_vaf_vs_avg_bq(df, min_depth=0, min_var=0, min_vaf=0, fig_file=None):
    select = (df['depth'] >= min_depth) & (df['count'] >= min_var)
    df = df[select]
    plt.plot(df['avg_basequality'], df['vaf'], 'o', color='black', alpha=0.6)
    plt.title('vaf vs avg_basequality')
    plt.xlabel('avg_basequality')
    plt.ylabel('vaf')

    if fig_file:
        plt.savefig(fig_file)

    plt.show()

In [33]:
def workflow(site_list, mapq=0, baseq=0, min_depth=0,min_vaf=0, remove_clip=True, bam_file_override=None):
    summary_results=[]
    for idx, sra in enumerate(SRAs):
        alignment_path=PRJ_OUT_PATH+sra
        if not os.path.isdir(alignment_path):
            pathlib.Path(alignment_path).mkdir(exist_ok=True)
        alignment_path=PRJ_OUT_PATH+sra+f'/{ALIGNER}/'
        assert os.path.isdir(alignment_path)
        if bam_file_override is not None:
            bam_file=f"{sra}{bam_file_override}.bam"
        else:
            bam_file = f"{sra}_{ALIGN_OPTS_NAME}_{ALIGN_NAME}_{FILTER}_{ALIGNER}_{BAM_PROC_STRING}.bam"
        if remove_clip:
            bam_file=remove_clipping(alignment_path,bam_file)
        assert os.path.isfile(alignment_path+bam_file)
        bam_readcount_output=run_bam_readcount(alignment_path, bam_file, site_list, mapq, baseq)
        tsv_strings=bam_readcount_results(sra, alignment_path+bam_readcount_output,min_depth=min_depth,min_vaf=min_vaf)
        summary_results.append(tsv_strings) 
        
    summary_results = [val for sublist in summary_results for val in sublist]
    sdata=[]
    for s in summary_results:
        st=s.split('\t')
        sdata.append(st)

    df = pd.DataFrame(sdata, columns=['sra', 'reference','position','ref_base','base','vaf','depth','count','avg_basequality','avg_pos_as_fraction'])
    df = df.astype( dtype={'sra':'str', 'reference':'str', 'position':'int', 'ref_base':'str','base':'str','vaf':'float', 'depth':'int',\
                           'count':'int', 'avg_basequality':'float', 'avg_pos_as_fraction':'float'})
    return df

### General Settings

In [34]:
MINIMAP2_PATH='~/apps/minimap2-2.24_x64-linux/'
SAMTOOLS_PATH='~/apps/samtools-1.14/bin/'
GATK_JAR='~/apps/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar'
BAMSTATS_JAR='~/apps/BAMStats-1.25/BAMStats-1.25.jar'
BAMDST_PATH='/mnt/1TB_0/Data/Code/external/bamdst/'
NGSUTILS_PATH='~/apps/ngsutils/'
BAM_READCOUNT_PATH='/mnt/1TB_0/Data/Code/external/bam-readcount/build/bin/'

In [35]:
ALIGNER='minimap2'
BAM_PROC_STRING='samtools_sorted'

REF_NAME='NC_045512_2_SARS_CoV_2_Wuahn-Hu-1_no_polyA.fa'
REF_PATH='/mnt/1TB_0/Data/fasta/complete_nucleotide/bwa_indexes/'
ALIGN_NAME=REF_NAME.split('.fa')[0]

FILTER='TrimGalore'

ALIGN_OPTS_NAME='x_sr_secondary_no'

SITE_LIST='NC_045512.2:8782-8782, NC_045512.2:28144-28144, NC_045512.2:29095-29095'


### BioProject specific

In [36]:
PRJ='PRJNA698267'
PRJ_OUT_PATH=f'/mnt/8TB_0/Data/Assembly/{PRJ}/'


SRAs=['SRR13615939','SRR13615941','SRR13615942','SRR13615948','SRR13615949','SRR13615950','SRR13615951','SRR13615952','SRR13615953','SRR13615954',\
      'SRR13615955','SRR13615956','SRR13615957','SRR13615958','SRR13615959','SRR13615960','SRR13615961','SRR13615962','SRR13615964','SRR13615965',\
      'SRR13615967','SRR13615969','SRR13615971','SRR13615976','SRR13615977','SRR13615982','SRR13615983','SRR13615984','SRR13615987','SRR13615988',\
      'SRR13615990','SRR13615991','SRR13615992','SRR13615994','SRR13615995','SRR13615996','SRR13615997','SRR13615998','SRR13616000','SRR13616001',\
      'SRR13616002','SRR13616003','SRR13616004','SRR13616010','SRR13616011','SRR13616012','SRR13616013','SRR13616014','SRR13616016','SRR13616017',\
      'SRR13616019','SRR13616021','SRR13616023','SRR13616024','SRR13616028']

df=workflow(SITE_LIST, mapq=0, baseq=0, min_depth=3,min_vaf=0.05, remove_clip=True)
df.to_csv(PRJ+'_lite_8782_28144_29095.csv')

In [37]:
PRJ='PRJNA698267'
PRJ_OUT_PATH=f'/mnt/8TB_0/Data/Assembly/{PRJ}/'

SRAs=['SRR13615940','SRR13615943','SRR13615944','SRR13615945','SRR13615946','SRR13615947','SRR13615963','SRR13615966','SRR13615968','SRR13615970',\
           'SRR13615972','SRR13615973','SRR13615974','SRR13615975','SRR13615978','SRR13615979','SRR13615980','SRR13615981','SRR13615985','SRR13615986',\
           'SRR13615989','SRR13615999','SRR13616005','SRR13616006','SRR13616007','SRR13616008','SRR13616009','SRR13616015','SRR13616018','SRR13616020',\
           'SRR13616022','SRR13616025','SRR13616026','SRR13616027','SRR13616029', 'SRR13615993']



df=workflow(SITE_LIST, mapq=0, baseq=0, min_depth=3,min_vaf=0.05, remove_clip=True)
df.to_csv(PRJ+'_8782_28144_29095.csv')